<a href="https://colab.research.google.com/github/illusoryTwin/FoR/blob/assig2/assig2/FoR_assig2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
class Manipulator:
    def __init__(self, l, m=None):
        self.l = l
        self.m = m
        self.q = None

    def set_config(self, q_):
        if self.q is None:
            self.q = q_ if isinstance(q_, list) else [q_]
        else:
            self.q.extend(q_ if isinstance(q_, list) else [q_])

    def get_config(self):
        return self.q

    def get_config_in_pi(self):
        return [str(q_ / np.pi) + "pi" for q_ in self.q]

l = [0.3, 0.5, 0.3, 0.5, 0.3, 0.1]
manipulator = Manipulator(l)

In [17]:
# Define rotation and translation functions
def rotate_z(theta):
    return np.array([[np.cos(theta), -np.sin(theta), 0, 0],
                     [np.sin(theta), np.cos(theta), 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

def rotate_x(theta):
    return np.array([[1, 0, 0, 0],
                     [0, np.cos(theta), -np.sin(theta), 0],
                     [0, np.sin(theta), np.cos(theta), 0],
                     [0, 0, 0, 1]])

def rotate_y(theta):
    return np.array([[np.cos(theta), 0, np.sin(theta), 0],
                     [0, 1, 0, 0],
                     [-np.sin(theta), 0, np.cos(theta), 0],
                     [0, 0, 0, 1]])

def translate_x(d):
    return np.array([[1, 0, 0, d],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

def translate_y(d):
    return np.array([[1, 0, 0, 0],
                     [0, 1, 0, d],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

def translate_z(d):
    return np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, d],
                     [0, 0, 0, 1]])

def validate_matrix(matrix):

    tolerance = 1e-12
    # Define a function to check if a value is close to zero within the tolerance
    def is_close_to_zero(value):
        return abs(value) < tolerance

    # Iterate through each element in the matrix
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            # If the element is close to zero, set it to zero
            if is_close_to_zero(matrix[i][j]):
                matrix[i][j] = 0


def retrieve_rot_matrix(matrix):
    return matrix[:3, :3]


class ForwardKinematicsSolver:
    def __init__(self, manipulator):
        self.manipulator = manipulator

    def forward_kinematics(self, q):
        theta1, theta2, theta3, theta4, theta5, theta6 = q
        l1, l2, l3, l4, l5, l6 = self.manipulator.l

        T_0 = np.identity(4)
        T_01 = rotate_z(theta1) @ translate_z(l1) @ rotate_z(np.pi/2) @ rotate_x(np.pi/2)
        T_12 = rotate_z(theta2) @ translate_x(l2)
        T_23 = rotate_z(theta3) @ translate_x(l3) @ rotate_y(np.pi/2) @ rotate_z(np.pi/2)
        T_34 = rotate_z(theta4) @ translate_z(l4) @ rotate_x(-np.pi/2) @ rotate_z(-np.pi)
        T_45 = rotate_z(theta5) @ translate_y(l5) @ rotate_x(-np.pi/2)
        T_56 = rotate_z(theta6) @ translate_z(l6)

        T = T_0 @ T_01 @ T_12 @ T_23 @ T_34 @ T_45 @ T_56

        # Validate the matrix
        validate_matrix(T)

        return T

    def fk_for_wr_center(self, q):
        theta1, theta2, theta3, theta4, theta5, theta6 = q
        l1, l2, l3, l4, l5, l6 = self.manipulator.l

        T_0 = np.identity(4)
        T_01 = rotate_z(theta1) @ translate_z(l1) @ rotate_z(np.pi/2) @ rotate_x(np.pi/2)
        T_12 = rotate_z(theta2) @ translate_x(l2)
        T_23_ = rotate_z(theta3) @ translate_x(l3+l4) @ rotate_y(np.pi/2) @ rotate_z(np.pi/2)

        T = T_0 @ T_01 @ T_12 @ T_23_

        # Validate the matrix
        validate_matrix(T)

        return T

    def fk_for_elbow(self, theta1, theta2, theta3):
        l1, l2, l3, l4, l5, l6 = self.manipulator.l

        T_0 = np.identity(4)
        T_01 = rotate_z(theta1) @ translate_z(l1) @ rotate_z(np.pi/2) @ rotate_x(np.pi/2)
        T_12 = rotate_z(theta2) @ translate_x(l2)
        T_23 = rotate_z(theta3) @ translate_x(l3) @ rotate_y(np.pi/2) @ rotate_z(np.pi/2)

        T = T_0 @ T_01 @ T_12 @ T_23

        # Validate the matrix
        validate_matrix(T)

        return T

In [18]:
# Create an instance of ForwardKinematicsSolver with your manipulator object
solver = ForwardKinematicsSolver(manipulator)

# Define the joint configuration q
q = [0, 0, 0, 0, 0, np.pi/6]

# Solve forward kinematics for the wrist center
solution1 = solver.fk_for_wr_center(q)

# Solve forward kinematics for the end effector
solution1_ee = solver.forward_kinematics(q)

print("Wrist Center Solution:")
print(solution1)

print("\nEnd Effector Solution:")
print(solution1_ee)

Wrist Center Solution:
[[0.  1.  0.  0. ]
 [0.  0.  1.  1.3]
 [1.  0.  0.  0.3]
 [0.  0.  0.  1. ]]

End Effector Solution:
[[-0.5       -0.8660254  0.         0.       ]
 [ 0.         0.         1.         1.7      ]
 [-0.8660254  0.5        0.         0.3      ]
 [ 0.         0.         0.         1.       ]]


In [19]:
# Corrected inverse kinematics

from numpy.linalg import inv


def inverse_kinematics(manipulator, ee_config, prev_state=None):
    l1, l2, l3, l4, l5, l6 = manipulator.l
    fk_solver = ForwardKinematicsSolver(manipulator)

    wr_c_to_ee = l5 + l6

    wrist_center = ee_config @ inv(translate_z(wr_c_to_ee))
    # wrist_center = forward_kinematics_for_wr_center(q)

    x = wrist_center[0][3]
    y = wrist_center[1][3]
    z = wrist_center[2][3]

    # q1
    try:
        q1 = np.arctan2(-x, y)

    except ZeroDivisionError:
        q1 = 0

    # q3
    q3_ = np.arccos(((z-l1)**2 + (x**2+y**2)- l2**2 - (l3+l4)**2)/(2*l2*(l3+l4)))

    if z < l1:
      q3_ = -q3_

    if q3_ == 0 or q3_ == np.pi:
        q3 = np.pi/2
        print("Singularity detected! Adjusted q3 to pi/2")
        if prev_state is not None:
            q3 = prev_state[2]  # Revert to previous state if available
    else:
        q3 = q3_

    # q2
    r = np.sqrt(x**2+y**2)
    s = z - l1
    q2 = np.arcsin(((l2+(l3+l4)*np.cos(q3))*s - (l3+l4)*np.sin(q3)*r)/(r**2+s**2))


    # q4
    R06 = retrieve_rot_matrix(ee_config) # ee_config is taken from the input
    R03 = retrieve_rot_matrix(fk_solver.fk_for_elbow(q1, q2, q3))

    # R03 = retrieve_rot_matrix(forward_kinematics_for_elbow(manipulator, q1, q2, q3))
    R03_T = R03.transpose()
    R36 = R03_T @ retrieve_rot_matrix(ee_config)
    validate_matrix(R36)
    R36 = retrieve_rot_matrix(R36)

    q4 = 0

    if R36[0][2] == 0:
        q4 = np.pi/2
    else:
        q4 = np.arctan(R36[1][2]/R36[0][2])

    # q5
    sin5 = 0
    q5_ = 0
    if np.sin(q4) == 0:
        q5_ = np.arccos(R36[2][2])
    elif np.sign(R36[1][2])/np.sign(np.sin(q4)) > 0:
        sin5 = np.sqrt(R36[0][2]**2 + R36[1][2]**2)
        q5_ = np.arctan2(sin5, R36[2][2])
    elif np.sign(R36[1][2])/np.sin(q4) < 0:
        sin5 = -np.sqrt(R36[0][2]**2 + R36[1][2]**2)
        q5_ = np.arctan2(sin5, R36[2][2])

    if q5_ == 0 or q5_ == np.pi:
        q5 = np.pi/2
        if prev_state is not None:
            q5 = prev_state[4]*np.sign(prev_state[4])  # Revert to previous state if available
            print("Singularity detected! Adjusted q5 to q5 of the previous state")
        else:
            print("Singularity detected! Adjusted q5 to pi/2")


    else:
        q5 = q5_

    # q6
    if R36[2][1] == R36[2][0] == 0:
        q6 = 0
    elif R36[2][0] == 0:
        q6 = np.pi/2
    else:
        q6 = np.arctan(-R36[2][1]/R36[2][0])

    q = (q1, q2, q3, q4, q5, q6)

    return q

def show_formatted_ik_solution(q):
    formatted_q = ["{:.6f}π".format(angle / np.pi) for angle in q]
    print(formatted_q)


In [22]:
# Define a manipulator instance
l = [0.3, 1, 0.3, 0.5, 0.3, 0.4]
manipulator = Manipulator(l)

fk_solver = ForwardKinematicsSolver(manipulator)

# Define the manipulator's configuration
# q = [np.pi/2, 0, np.pi/12, 0, np.pi/3, np.pi/6] # error here
q = [np.pi/4, np.pi/12, np.pi/12, np.pi/12, -np.pi/12, np.pi/6]

config1 = fk_solver.forward_kinematics(q)

solution1 = inverse_kinematics(manipulator, config1)
show_formatted_ik_solution(solution1)

['0.250000π', '0.083333π', '0.083333π', '0.083333π', '-0.083333π', '0.166667π']


# Some tests

In [4]:
import numpy as np

# Define rotation and translation functions
def rotate_z(theta):
    return np.array([[np.cos(theta), -np.sin(theta), 0, 0],
                     [np.sin(theta), np.cos(theta), 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

def rotate_x(theta):
    return np.array([[1, 0, 0, 0],
                     [0, np.cos(theta), -np.sin(theta), 0],
                     [0, np.sin(theta), np.cos(theta), 0],
                     [0, 0, 0, 1]])

def rotate_y(theta):
    return np.array([[np.cos(theta), 0, np.sin(theta), 0],
                     [0, 1, 0, 0],
                     [-np.sin(theta), 0, np.cos(theta), 0],
                     [0, 0, 0, 1]])

def translate_x(d):
    return np.array([[1, 0, 0, d],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

def translate_y(d):
    return np.array([[1, 0, 0, 0],
                     [0, 1, 0, d],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

def translate_z(d):
    return np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, d],
                     [0, 0, 0, 1]])

def validate_matrix(matrix):

    tolerance = 1e-12
    # Define a function to check if a value is close to zero within the tolerance
    def is_close_to_zero(value):
        return abs(value) < tolerance

    # Iterate through each element in the matrix
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            # If the element is close to zero, set it to zero
            if is_close_to_zero(matrix[i][j]):
                matrix[i][j] = 0


def retrieve_rot_matrix(matrix):
    return matrix[:3, :3]

# ============================================================

# Define the forward kinematics function for the manipulator
def forward_kinematics(manipulator, q):
    theta1, theta2, theta3, theta4, theta5, theta6 = q
    l1, l2, l3, l4, l5, l6 = manipulator.l

    T_0 = np.identity(4)
    T_01 = rotate_z(theta1) @ translate_z(l1) @ rotate_z(np.pi/2) @ rotate_x(np.pi/2)
    T_12 = rotate_z(theta2) @ translate_x(l2)
    T_23 = rotate_z(theta3) @ translate_x(l3) @ rotate_y(np.pi/2) @ rotate_z(np.pi/2)
    T_34 = rotate_z(theta4) @ translate_z(l4) @ rotate_x(-np.pi/2) @ rotate_z(-np.pi)
    T_45 = rotate_z(theta5) @ translate_y(l5) @ rotate_x(-np.pi/2)
    T_56 = rotate_z(theta6) @ translate_z(l6)

    T = T_0 @ T_01 @ T_12 @ T_23 @ T_34 @ T_45 @ T_56

    # Validate the matrix
    validate_matrix(T)

    return T

# Define the forward kinematics function for the center
def forward_kinematics_for_wr_center(manipulator, q):
    theta1, theta2, theta3, theta4, theta5, theta6 = q
    l1, l2, l3, l4, l5, l6 = manipulator.l

    T_0 = np.identity(4)
    T_01 = rotate_z(theta1) @ translate_z(l1) @ rotate_z(np.pi/2) @ rotate_x(np.pi/2)
    T_12 = rotate_z(theta2) @ translate_x(l2)
    T_23_ = rotate_z(theta3) @ translate_x(l3+l4) @ rotate_y(np.pi/2) @ rotate_z(np.pi/2)

    T = T_0 @ T_01 @ T_12 @ T_23_

    # Validate the matrix
    validate_matrix(T)

    return T

def forward_kinematics_for_elbow(manipulator, theta1, theta2, theta3):
    l1, l2, l3, l4, l5, l6 = manipulator.l

    T_0 = np.identity(4)
    T_01 = rotate_z(theta1) @ translate_z(l1) @ rotate_z(np.pi/2) @ rotate_x(np.pi/2)
    T_12 = rotate_z(theta2) @ translate_x(l2)
    T_23 = rotate_z(theta3) @ translate_x(l3) @ rotate_y(np.pi/2) @ rotate_z(np.pi/2)

    T = T_0 @ T_01 @ T_12 @ T_23

    # Validate the matrix
    validate_matrix(T)

    return T


# Calculate the forward kinematics
q = [0, 0, 0, 0, 0, np.pi/6]
solution1 = forward_kinematics_for_wr_center(manipulator, q)
solution1_ee = forward_kinematics(manipulator, q)
print(solution1_ee)


[[-0.5       -0.8660254  0.         0.       ]
 [ 0.         0.         1.         1.7      ]
 [-0.8660254  0.5        0.         0.3      ]
 [ 0.         0.         0.         1.       ]]
